<a href="https://colab.research.google.com/github/Marcel0609/Thesis-Code/blob/main/Location_%26_Lead_Time_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 15/10/2025

!pip install openrouteservice
import os
import re
import time
import math
import random
import pandas as pd
import openrouteservice
from tqdm import tqdm
from google.colab import drive

ENFORCE_SYMMETRY = False     # Optional, bisa kita ganti jadi true kalo mau symetry
RATE_LIMIT_SLEEP = 2.6
RATE_LIMIT_JITTER = 0.7
SECOND_MATRIX_BATCH = True
NO_ROUTE_LABEL = "no land routes available"

drive.mount("/content/drive", force_remount=True)

folder_path = '/content/drive/MyDrive/Colab Notebooks/Skripsi/'
file_name = 'trialcode.xlsx'
full_path = os.path.join(folder_path, file_name)
os.chdir(folder_path)

print("Current directory")
!pwd
print("Files in directory")
!ls

print("\n" + "="*60)
print("Loading data from excel")
print("="*60)

master_distance = pd.read_excel(full_path, sheet_name='Master_Distance')
distance_matrix_template = pd.read_excel(full_path, sheet_name='Distance_Matrix', index_col=0)

print(f"Master_Distance loaded: {len(master_distance)} baris")
print(f"Distance_Matrix template loaded: {distance_matrix_template.shape}")

required_cols = {'Origin', 'Destination', 'Distance_km'}
missing_cols = required_cols - set(master_distance.columns)
if missing_cols:
    raise ValueError(f"Missing columns: {missing_cols}")

print("\n" + "="*60)
print("Loading coordinates")
print("="*60)

coordinate_data = {
    'HUB BTL': (116.00, -3.41), 'BTL': (116.00, -3.41), 'HUB BLP': (116.85, -1.24),
    'BLP': (116.96, -1.22), 'WHSBLP': (116.96, -1.22), 'KMSIBLP': (116.96, -1.22),
    'HUB BJM': (114.68, -3.40), 'SPT': (112.88, -2.52), 'WHSSPT': (112.88, -2.52),
    'HUB TJG': (115.43, -2.42), 'SDJ': (115.61, -3.71), 'KEL': (115.89, -3.00),
    'STI': (115.60, -3.71), 'HUB SGT': (117.52, 0.55), 'BDI': (117.50, 0.54),
    'MLW': (115.37, -0.46), 'SPR': (117.09, -0.30), 'SMD': (117.10, -0.53),
    'WHS LJN': (117.06, -0.64), 'LJN': (117.06, -0.64), 'KMSI BLP': (116.96, -1.22),
    'RTU': (115.08, -3.12), 'RTT': (115.23, -2.95), 'TBG': (115.62, -3.60),
    'TJR': (117.46, 2.16), 'BKJ': (115.87, -1.92), 'BNE': (117.50, 0.13),
    'SKH': (116.71, 1.06), 'GRM': (115.40, -3.00), 'SDU': (115.38, -3.79),
    'DMI': (115.85, -0.31), 'JBY': (117.10, -0.17), 'WHS SGA': (117.2043, -0.6441),
    'SGA': (117.2043, -0.6441),  # <-- Add this line
    'TTN': (114.54, -0.66), 'BIU': (115.89, -1.82), 'BNT': (117.50, 0.13),
    'BGP': (117.59, 0.82), 'SKL': (116.71, 1.06), 'BEK': (115.95, -1.72), 'BGL': (118.05, 1.05)
}

def norm(name: str) -> str:
    if pd.isna(name):
        return ""
    return re.sub(r"\W+", "", str(name)).upper()

coord_by_norm = {norm(k): v for k, v in coordinate_data.items()}

def get_coords(label: str):
    if label in coordinate_data:
        return coordinate_data[label]
    return coord_by_norm.get(norm(label), None)

# Filter nilai NaN dari indeks sebelum membuat matrix_locations.
# Filter out NaN values from the index before creating matrix_locations
matrix_locations = [loc for loc in distance_matrix_template.index if pd.notna(loc)]
missing_coords = [loc for loc in matrix_locations if get_coords(loc) is None]
if missing_coords:
    suggestions = {}
    for m in missing_coords:
        mn = norm(m)
        cands = [k for k in coordinate_data if norm(k) == mn or mn in norm(k) or norm(k) in mn]
        suggestions[m] = cands[:3]
    raise KeyError(
        "Some labels dont have coordinates"
        f"Without coordinates: {missing_coords}\nCandidates: {suggestions}"
    )

print("\n" + "="*60)
print("Fixing data Distance_km")
print("="*60)

def to_float_km(x):
    if pd.isna(x):
        return None
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip().lower()
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[^\d,.\-]", "", s)
    if "," in s and "." in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s:
        s = s.replace(",", ".")
    else:
        if s.count(".") > 1:
            parts = s.split(".")
            s = "".join(parts[:-1]) + "." + parts[-1]
    try:
        return float(s)
    except ValueError:
        return None

master_distance["Distance_km_clean"] = master_distance["Distance_km"].apply(to_float_km)

md_lookup = {
    (str(o), str(d)): float(km)
    for o, d, km in master_distance[["Origin", "Destination", "Distance_km_clean"]]
        .itertuples(index=False, name=None)
    if pd.notna(km)
}
md_lookup_norm = {
    (norm(o), norm(d)): float(km)
    for o, d, km in master_distance[["Origin", "Destination", "Distance_km_clean"]]
        .itertuples(index=False, name=None)
    if pd.notna(km)
}

print(f"Master lookup ready: exact={len(md_lookup)}, normalized={len(md_lookup_norm)}")

print("\n" + "="*60)
print("SETUP ORS, SNAP-TO-ROAD, DAN UTILITAS")
print("="*60)

api_key = os.getenv(
    "ORS_KEY",
    "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImJjZmM3NDViYTNjMjRhMzBhMTAyMjI1ZjlkZGU3MTk4IiwiaCI6Im11cm11cjY0In0="
)
client = openrouteservice.Client(key=api_key)

def sleep_backoff():
    time.sleep(RATE_LIMIT_SLEEP + random.uniform(0, RATE_LIMIT_JITTER))

def snap_to_road(coord):
    try:
        res = client.nearest(coordinates=[coord], number=1)
        snapped = tuple(res["features"][0]["geometry"]["coordinates"])
        return snapped
    except Exception:
        return coord

def directions_distance_km(a, b):
    routes = client.directions(
        coordinates=[list(a), list(b)],
        profile='driving-car',
        format_out='json',
        units='km'
    )
    return float(routes['routes'][0]['summary']['distance'])

print("ORS ready.")

print("\n" + "="*60)
print("snap coordinate to the nearest road network")
print("="*60)

orig_coords = [get_coords(loc) for loc in matrix_locations]
snapped_coords = []
for c in tqdm(orig_coords, desc="Snap-to-road"):
    snapped_coords.append(snap_to_road(c))

n = len(matrix_locations)

road_matrix = pd.DataFrame(index=matrix_locations, columns=matrix_locations, dtype=object)

for i in range(n):
    road_matrix.iat[i, i] = 0.0

for i, o in enumerate(matrix_locations):
    for j, d in enumerate(matrix_locations):
        if i == j:
            continue
        if (o, d) in md_lookup:
            road_matrix.iat[i, j] = md_lookup[(o, d)]
        elif (norm(o), norm(d)) in md_lookup_norm:
            road_matrix.iat[i, j] = md_lookup_norm[(norm(o), norm(d))]
print("\n" + "="*60)
print("Pulling driving distance matrix")
print("="*60)

try:
    matrix_res = client.distance_matrix(
        locations=snapped_coords,
        profile='driving-car',
        metrics=['distance'],
        resolve_locations=False,
        sources=list(range(n)),
        destinations=list(range(n))
    )
    raw_dist = matrix_res.get('distances')
except openrouteservice.exceptions.ApiError as e:
    raw_dist = None
    print("Failed pulling matrix ORS:", e)

filled_from_ors = 0
if raw_dist is not None:
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if road_matrix.iat[i, j] is not None and pd.notna(road_matrix.iat[i, j]):
                continue
            val = raw_dist[i][j]
            if val is None:
                continue
            km = round(float(val) / 1000.0, 2)
            road_matrix.iat[i, j] = km
            filled_from_ors += 1

print(f"ORS matrix pairs: {filled_from_ors}")

if SECOND_MATRIX_BATCH:
    print("\n" + "="*60)
    print("Second trial matrix (subset pairs NaN, driving-car)")
    print("="*60)
    na_idx = [(i, j) for i in range(n) for j in range(n)
              if i != j and (road_matrix.iat[i, j] is None or pd.isna(road_matrix.iat[i, j]))]
    if na_idx:
        srcs = sorted(set(i for i, _ in na_idx))
        dsts = sorted(set(j for _, j in na_idx))
        try:
            sub_res = client.distance_matrix(
                locations=snapped_coords,
                profile='driving-car',
                metrics=['distance'],
                resolve_locations=False,
                sources=srcs,
                destinations=dsts
            )
            sub_dist = sub_res.get('distances')
            if sub_dist is not None:
                for si, i in enumerate(srcs):
                    for dj, j in enumerate(dsts):
                        val = sub_dist[si][dj]
                        if val is None:
                            continue
                        if road_matrix.iat[i, j] is None or pd.isna(road_matrix.iat[i, j]):
                            road_matrix.iat[i, j] = round(float(val) / 1000.0, 2)
        except openrouteservice.exceptions.ApiError as e:
            print("Trial failed:", e)

print("\n" + "="*60)
print("FALLBACK DIRECTIONS (driving-car, upper triangle)")
print("="*60)

fallback_success = 0
fallback_fail = 0

for i in tqdm(range(n), desc="Directions fallback"):
    for j in range(i+1, n):
        if road_matrix.iat[i, j] is not None and pd.notna(road_matrix.iat[i, j]):
            if ENFORCE_SYMMETRY and (road_matrix.iat[j, i] is None or pd.isna(road_matrix.iat[j, i])):
                road_matrix.iat[j, i] = road_matrix.iat[i, j]
            continue

        a = snapped_coords[i]
        b = snapped_coords[j]
        try:
            time.sleep(RATE_LIMIT_SLEEP + random.uniform(0, RATE_LIMIT_JITTER))
            km = round(directions_distance_km(a, b), 2)
            road_matrix.iat[i, j] = km
            if ENFORCE_SYMMETRY:
                road_matrix.iat[j, i] = km
            fallback_success += 1
        except openrouteservice.exceptions.ApiError:
            road_matrix.iat[i, j] = NO_ROUTE_LABEL
            if ENFORCE_SYMMETRY and (road_matrix.iat[j, i] is None or pd.isna(road_matrix.iat[j, i])):
                road_matrix.iat[j, i] = NO_ROUTE_LABEL
            fallback_fail += 1
        except Exception:
            road_matrix.iat[i, j] = NO_ROUTE_LABEL
            if ENFORCE_SYMMETRY and (road_matrix.iat[j, i] is None or pd.isna(road_matrix.iat[j, i])):
                road_matrix.iat[j, i] = NO_ROUTE_LABEL
            fallback_fail += 1

print(f"Fallback directions success: {fallback_success} | marked no-route: {fallback_fail}")

print("\n" + "="*60)
print("Saving in trialcode.xlsx")
print("="*60)

for i in range(n):
    road_matrix.iat[i, i] = 0.0

road_matrix = road_matrix.where(pd.notna(road_matrix), other=NO_ROUTE_LABEL)

total_pairs = n * n - n
no_route_pairs = int((pd.DataFrame(road_matrix.values) == NO_ROUTE_LABEL).sum().sum())
summary_df = pd.DataFrame(
    {
        "Metric": ["Locations", "Total off-diagonal pairs", "Filled via ORS matrix",
                   "Fallback directions success", "Marked no-route"],
        "Value":  [n, total_pairs, filled_from_ors, fallback_success, no_route_pairs],
    }
)

no_route_list = []
for i, o in enumerate(matrix_locations):
    for j, d in enumerate(matrix_locations):
        if i == j:
            continue
        if road_matrix.iat[i, j] == NO_ROUTE_LABEL:
            no_route_list.append({"Origin": o, "Destination": d})

no_route_df = pd.DataFrame(no_route_list) if no_route_list else pd.DataFrame(columns=["Origin","Destination"])

from pandas import ExcelWriter

with ExcelWriter(full_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    road_matrix.to_excel(writer, sheet_name="Distance_Matrix result")
    summary_df.to_excel(writer, sheet_name="Road_Summary", index=False)
    no_route_df.to_excel(writer, sheet_name="NoRoute_Pairs", index=False)

print(f"Savubg: {full_path}")
print("   - Sheet: RoadOnly_Matrix, Road_Summary, NoRoute_Pairs")

print("Preview 5x5 (atas-kiri) ")
print(road_matrix.iloc[:5, :5])

print("Finish")

Mounted at /content/drive
Current directory
/content/drive/MyDrive/Colab Notebooks/Skripsi
Files in directory
Distance_Matrix_Corrected.csv		      Distance_Matrix_RoadOnly.csv
Distance_Matrix_Corrected_FINAL.csv	      Distance_Matrix_RoadOnly.xlsx
Distance_Matrix_Fallback_Haversine_Pairs.csv  Kalimantan_Locations_Map.html
Distance_Matrix_Filled.csv		      trialcode.xlsx
Distance_Matrix_FINAL.csv

Loading data from excel
Master_Distance loaded: 93 baris
Distance_Matrix template loaded: (34, 34)

Loading coordinates

Fixing data Distance_km
Master lookup ready: exact=87, normalized=87

SETUP ORS, SNAP-TO-ROAD, DAN UTILITAS
ORS ready.

snap coordinate to the nearest road network


Snap-to-road: 100%|██████████| 34/34 [00:00<00:00, 166596.19it/s]


Pulling driving distance matrix


ORS matrix pairs: 928

Second trial matrix (subset pairs NaN, driving-car)

FALLBACK DIRECTIONS (driving-car, upper triangle)


Directions fallback: 100%|██████████| 34/34 [03:20<00:00,  5.88s/it]


Fallback directions success: 0 | marked no-route: 64

Saving in trialcode.xlsx
Savubg: /content/drive/MyDrive/Colab Notebooks/Skripsi/trialcode.xlsx
   - Sheet: RoadOnly_Matrix, Road_Summary, NoRoute_Pairs
Preview 5x5 (atas-kiri) 
        HUB BLP     KEL HUB BTL HUB SGT     BDI
HUB BLP     0.0   328.0  503.17   272.0   389.0
KEL       515.0     0.0   140.0  682.41  681.77
HUB BTL  503.16   130.0     0.0  718.56  717.92
HUB SGT   276.0  682.49  718.67     0.0    16.9
BDI       277.0  681.88  718.05     1.8     0.0
Finish


In [ ]:
# Visualisasi Peta
# 16/10/2025

!pip install folium --quiet

import folium
from IPython.display import display
import os

points = []
in_matrix_set = set(matrix_locations) if "matrix_locations" in globals() else set()
for name, (lon, lat) in coordinate_data.items():
    points.append({
        "name": name,
        "lat": float(lat),
        "lon": float(lon),
        "is_hub": name.strip().upper().startswith("HUB")
    })
center_lat = sum(p["lat"] for p in points) / len(points)
center_lon = sum(p["lon"] for p in points) / len(points)

m = folium.Map(location=[center_lat, center_lon], zoom_start=6, tiles="OpenStreetMap", control_scale=True)

# Tambahkan titik pada peta
for p in points:
    # warna merah untuk hub BJM dan BLP, sisanya biru
    if p["name"].strip().upper() == "HUB BJM" or p["name"].strip().upper() == "HUB BLP":
        color = "red"
    else:
        color = "blue" if p["is_hub"] else "blue"  # warna biru untuk lainnya

    folium.Marker(
        location=[p["lat"], p["lon"]],
        popup=folium.Popup(
            f"<b>{p['name']}</b><br>Lat: {p['lat']:.6f}<br>Lon: {p['lon']:.6f}", max_width=250
        ),
        tooltip=f"{p['name']} ({p['lat']:.4f}, {p['lon']:.4f})",
        icon=folium.Icon(color=color)
    ).add_to(m)

# Menyimpan peta
map_path = os.path.join(folder_path, "Kalimantan_Locations_Map.html")
m.save(map_path)
print("✓ Map saved to:", map_path)
m  # Selesai


✓ Map saved to: /content/drive/MyDrive/Colab Notebooks/Skripsi/Kalimantan_Locations_Map.html


In [ ]:
import math
import pandas as pd
from pandas import ExcelWriter

AVG_SPEED_KMPH            = 45.0   # avg trucking speed
ORIGIN_HANDLING_HOURS     = 2.0    # loading + paperwork stuff pas di origin atau starting pointny
DEST_HANDLING_HOURS       = 2.0    # unloading + paperwork stuff pas di destination
ROUNDING_INCREMENT_HOURS  = 0.5    # round up to nearest 0.5h (bisa diganti jadi 1.0 for whole hours)
NO_ROUTE_LABEL            = NO_ROUTE_LABEL if "NO_ROUTE_LABEL" in globals() else "no land routes available"

assert "road_matrix" in globals(), "road_matrix not found. Run the distance code first."
assert "matrix_locations" in globals(), "matrix_locations not found."
assert "full_path" in globals(), "full_path not found (Excel path)."


def round_up_to_increment(x, inc):
    return math.ceil(x / inc) * inc


idx = list(matrix_locations)
lead_hours = pd.DataFrame(index=idx, columns=idx, dtype=object)
lead_days  = pd.DataFrame(index=idx, columns=idx, dtype=object)

for i, o in enumerate(idx):
    for j, d in enumerate(idx):
        if i == j:
            lead_hours.iat[i, j] = 0.0
            lead_days.iat[i, j]  = 0.0
            continue

        cell = road_matrix.iat[i, j]

        if isinstance(cell, str):
            lead_hours.iat[i, j] = NO_ROUTE_LABEL
            lead_days.iat[i, j]  = NO_ROUTE_LABEL
            continue

        try:
            dist_km = float(cell)
        except Exception:
            lead_hours.iat[i, j] = NO_ROUTE_LABEL
            lead_days.iat[i, j]  = NO_ROUTE_LABEL
            continue

        drive_hours = dist_km / max(AVG_SPEED_KMPH, 1e-6)
        raw_hours   = drive_hours + ORIGIN_HANDLING_HOURS + DEST_HANDLING_HOURS
        rounded_h   = round_up_to_increment(raw_hours, ROUNDING_INCREMENT_HOURS)

        lead_hours.iat[i, j] = float(f"{rounded_h:.2f}")
        lead_days.iat[i, j]  = float(f"{(rounded_h / 24.0):.2f}")


total_pairs  = len(idx) * len(idx) - len(idx)
no_route_cnt = int((pd.DataFrame(lead_hours.values) == NO_ROUTE_LABEL).sum().sum())
numeric_pairs = total_pairs - no_route_cnt

mean_hours = (
    pd.DataFrame(lead_hours.values)
      .apply(pd.to_numeric, errors="coerce")
      .stack()
      .loc[lambda s: s.index.get_level_values(0) != s.index.get_level_values(1)]
      .mean()
    if numeric_pairs > 0 else float("nan")
)

params = pd.DataFrame({
    "Parameter": [
        "AVG_SPEED_KMPH",
        "ORIGIN_HANDLING_HOURS",
        "DEST_HANDLING_HOURS",
        "ROUNDING_INCREMENT_HOURS",
        "Locations",
        "Total off-diagonal pairs",
        "No-route pairs",
        "Lead time mean (hours) on routed lanes",
    ],
    "Value": [
        AVG_SPEED_KMPH,
        ORIGIN_HANDLING_HOURS,
        DEST_HANDLING_HOURS,
        ROUNDING_INCREMENT_HOURS,
        len(idx),
        total_pairs,
        no_route_cnt,
        None if math.isnan(mean_hours) else round(float(mean_hours), 2)
    ]
})

with ExcelWriter(full_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    lead_hours.to_excel(writer, sheet_name="LeadTime_Hours")
    lead_days.to_excel(writer,  sheet_name="LeadTime_Days")
    params.to_excel(writer,     sheet_name="LeadTime_Params", index=False)

print("✓ Lead time sheets written to:", full_path)
print("   - LeadTime_Hours")
print("   - LeadTime_Days")
print("   - LeadTime_Params")

print("\nPreview (LeadTime_Hours, top-left 5x5):")
print(lead_hours.iloc[:5, :5])


✓ Lead time sheets written to: /content/drive/MyDrive/Colab Notebooks/Skripsi/trialcode.xlsx
   - LeadTime_Hours
   - LeadTime_Days
   - LeadTime_Params

Preview (LeadTime_Hours, top-left 5x5):
        HUB BLP   KEL HUB BTL HUB SGT   BDI
HUB BLP     0.0  11.5    15.5    10.5  13.0
KEL        15.5   0.0     7.5    19.5  19.5
HUB BTL    15.5   7.0     0.0    20.0  20.0
HUB SGT    10.5  19.5    20.0     0.0   4.5
BDI        10.5  19.5    20.0     4.5   0.0
